# Pandas (Parte 3)

In [50]:
import pandas as pd
import numpy as np

In [2]:
locacion_datos = "https://otorongo.club/2021/json/ingresos/"

cong = pd.read_json(locacion_datos)

In [7]:
cong_1 = cong[['dni', 'total_ingreso']]

cong_2 = cong[['dni', 'partido']]

solo_vn = cong_2.loc[cong_2['partido'] == 'VICTORIA NACIONAL', :]

Cómo es un outer merge? 

In [11]:
resultado_merge_vn = pd.merge(cong_1,
                     solo_vn,
                    left_on = 'dni',
                    right_on = 'dni',
                    how = 'outer')

resultado_merge_vn
#resultado_merge_vn.loc[resultado_merge_vn['partido'] != 'VICTORIA NACIONAL']
#resultado_merge_vn.loc[resultado_merge_vn['partido'] == 'VICTORIA NACIONAL']

,dni,total_ingreso,partido
0,27428781,57560800,VICTORIA NACIONAL
1,7246887,12428083,NaN
2,7818712,10998927,NaN
3,17903382,9571448,NaN
4,19873483,7740000,NaN
...,...,...,...
3311,44720712,0,NaN
3312,8505376,0,NaN
3313,48855918,0,NaN
3314,70013249,0,NaN


Cómo es un left merge? Y un right merge? 


In [14]:
resultado_merge_vn = pd.merge(cong_1,
                     solo_vn,
                    left_on = 'dni',
                    right_on = 'dni',
                    how = 'left')
#                    how = 'right')

resultado_merge_vn

,dni,total_ingreso,partido
0,27428781,57560800,VICTORIA NACIONAL
1,7246887,12428083,NaN
2,7818712,10998927,NaN
3,17903382,9571448,NaN
4,19873483,7740000,NaN
...,...,...,...
3311,44720712,0,NaN
3312,8505376,0,NaN
3313,48855918,0,NaN
3314,70013249,0,NaN


### Uniendo varias bases de datos verticalmente (append)

En pandas, este método se llama concat. 

In [22]:
df_partidos = {}
for cat in cong['partido'].unique():
    df = cong.loc[cong['partido'] == cat]
    df_partidos[cat] = df

In [29]:
df_fp = df_partidos['FUERZA POPULAR']
df_vn = df_partidos['VICTORIA NACIONAL']

In [31]:
df_append = pd.concat([df_vn,df_fp])

In [33]:
#df_append

In [35]:
df_append = pd.concat([df_partidos['FUERZA POPULAR'], df_partidos['VICTORIA NACIONAL']])

### Limpieza de datos (parte 2)

Eliminando columnas

In [16]:
#Creando (y eliminando) una columna que refleja un mal cálculo
cong['calculo_mal_hecho'] = cong.eval('ingreso_publico + ingreso_privado')

In [20]:
cong.drop(columns = 'calculo_mal_hecho', inplace = True)


Identificando (y eliminando) duplicados:

In [36]:
nueva_df = pd.concat([cong, cong])

In [38]:
nueva_df.head(5)

,nombre,dni,partido,total_ingreso,ingreso_publico,ingreso_privado,renta_publico,renta_privado,otro_ingreso_publico,otro_ingreso_privado
0,SEMPERTEGUI ILATOMA GILBERTO,27428781,VICTORIA NACIONAL,57560800,2710800,14400000,0,0,0,40450000
1,LUNA GALVEZ JOSE LEON,7246887,PODEMOS PERU,12428083,0,1732402,0,0,0,10695681
2,KUOMAN SAAVEDRA DAVID,7818712,RENOVACION POPULAR,10998927,0,92096,0,0,0,10906831
3,ACUÑA PERALTA CESAR,17903382,ALIANZA PARA EL PROGRESO,9571448,0,3063448,0,0,0,6508000
4,OTIVO MANTURANO TITO ALBERTICO,19873483,RENACIMIENTO UNIDO NACIONAL,7740000,3060000,0,0,1800000,0,2880000


In [40]:
nueva_df.nunique()

nombre                  3316
dni                     3316
partido                   22
total_ingreso           1480
ingreso_publico          655
ingreso_privado          477
renta_publico            247
renta_privado            464
otro_ingreso_publico      84
otro_ingreso_privado     295
dtype: int64

In [41]:
nueva_df.nunique(dropna=False)

nombre                  3316
dni                     3316
partido                   22
total_ingreso           1480
ingreso_publico          655
ingreso_privado          477
renta_publico            247
renta_privado            464
otro_ingreso_publico      84
otro_ingreso_privado     295
dtype: int64

In [42]:
## Cuales son las dimensiones de nuestra dataframe si eliminamos las observaciones duplicadas
nueva_df.drop_duplicates().shape

(3316, 10)

In [43]:
nueva_df.shape

(6632, 10)

In [46]:
sin_duplicados = nueva_df.drop_duplicates(subset='dni')
sin_duplicados.head(5)

,nombre,dni,partido,total_ingreso,ingreso_publico,ingreso_privado,renta_publico,renta_privado,otro_ingreso_publico,otro_ingreso_privado
0,SEMPERTEGUI ILATOMA GILBERTO,27428781,VICTORIA NACIONAL,57560800,2710800,14400000,0,0,0,40450000
1,LUNA GALVEZ JOSE LEON,7246887,PODEMOS PERU,12428083,0,1732402,0,0,0,10695681
2,KUOMAN SAAVEDRA DAVID,7818712,RENOVACION POPULAR,10998927,0,92096,0,0,0,10906831
3,ACUÑA PERALTA CESAR,17903382,ALIANZA PARA EL PROGRESO,9571448,0,3063448,0,0,0,6508000
4,OTIVO MANTURANO TITO ALBERTICO,19873483,RENACIMIENTO UNIDO NACIONAL,7740000,3060000,0,0,1800000,0,2880000


In [47]:
sin_duplicados_2 = nueva_df.drop_duplicates(subset=['dni','partido'])


### Manejando valores perdidos (missing)

A continuación veremos cómo manejar valores perdidos (missing). Veremos 3 métodos en especial:
- ` isna`
- `dropna`
- `fillna`

In [80]:
cong_m = cong.copy(deep = True)

In [81]:
cong_m.replace({0:np.nan}, inplace = True)

In [82]:
cong_m.isna().head(5)

,nombre,dni,partido,total_ingreso,ingreso_publico,ingreso_privado,renta_publico,renta_privado,otro_ingreso_publico,otro_ingreso_privado
0,False,False,False,False,False,False,True,True,True,False
1,False,False,False,False,True,False,True,True,True,False
2,False,False,False,False,True,False,True,True,True,False
3,False,False,False,False,True,False,True,True,True,False
4,False,False,False,False,False,True,True,False,True,False


In [83]:
cong_m.isna().sum()

nombre                     0
dni                        0
partido                    0
total_ingreso            570
ingreso_publico         2384
ingreso_privado         2524
renta_publico           2958
renta_privado           2230
otro_ingreso_publico    3221
otro_ingreso_privado    2735
dtype: int64

In [62]:
cong_m.isna().mean().round(4)*100

nombre                   0.00
dni                      0.00
partido                  0.00
total_ingreso           17.19
ingreso_publico         71.89
ingreso_privado         76.12
renta_publico           89.20
renta_privado           67.25
otro_ingreso_publico    97.14
otro_ingreso_privado    82.48
dtype: float64

In [78]:
## Aquí vemos cuales observaciones se eliminarían 
cong_m.dropna().shape

(3316, 10)

In [77]:
cong_m.dropna(subset=['ingreso_publico', 'ingreso_privado']).shape

(3316, 10)

In [86]:
#cong_m.fillna('hola')

In [87]:
dict_fill = {'total_ingreso': 0, 'ingreso_publico': 'hola'}
#cong_m.fillna(dict_fill)

In [88]:
med_fill = cong_m.median(numeric_only=True)
med_fill

dni                     23921498.0
total_ingreso              33000.0
ingreso_publico            33385.5
ingreso_privado            24000.0
renta_publico              24000.0
renta_privado              19000.0
otro_ingreso_publico       28080.0
otro_ingreso_privado       24000.0
dtype: float64

In [90]:
med_fill = cong_m.median(numeric_only=True)

cong_m.fillna(med_fill)

,nombre,dni,partido,total_ingreso,ingreso_publico,ingreso_privado,renta_publico,renta_privado,otro_ingreso_publico,otro_ingreso_privado
0,SEMPERTEGUI ILATOMA GILBERTO,27428781.0,VICTORIA NACIONAL,57560800.0,2710800.0,14400000.0,NaN,NaN,NaN,40450000.0
1,LUNA GALVEZ JOSE LEON,7246887.0,PODEMOS PERU,12428083.0,NaN,1732402.0,NaN,NaN,NaN,10695681.0
2,KUOMAN SAAVEDRA DAVID,7818712.0,RENOVACION POPULAR,10998927.0,NaN,92096.0,NaN,NaN,NaN,10906831.0
3,ACUÑA PERALTA CESAR,17903382.0,ALIANZA PARA EL PROGRESO,9571448.0,NaN,3063448.0,NaN,NaN,NaN,6508000.0
4,OTIVO MANTURANO TITO ALBERTICO,19873483.0,RENACIMIENTO UNIDO NACIONAL,7740000.0,3060000.0,NaN,NaN,1800000.0,NaN,2880000.0
...,...,...,...,...,...,...,...,...,...,...
3311,GUTIERREZ RIVERA SUSANA CAROLINA,44720712.0,PODEMOS PERU,33000.0,NaN,NaN,NaN,NaN,NaN,NaN
3312,CISNEROS CALDERON ANIBAL HUGO,8505376.0,RENACIMIENTO UNIDO NACIONAL,33000.0,NaN,NaN,NaN,NaN,NaN,NaN
3313,GONZALES FLORES LIZBETH ROXANA,48855918.0,PODEMOS PERU,33000.0,NaN,NaN,NaN,NaN,NaN,NaN
3314,MELGAR GALVEZ FIORELA JUSETTE,70013249.0,PODEMOS PERU,33000.0,NaN,NaN,NaN,NaN,NaN,NaN


### Haciendo un reshape de los datos (volver los datos de wide a long, y viceversa).



In [106]:
categ_labels = ['cat_1', 'cat_2', 'cat_3', 'cat_4']
categ_bins = [-1, 10000, 50000, 100000, 200000000]
cong['cat_ingreso'] = pd.cut(cong['total_ingreso'],
                              bins = categ_bins, labels = categ_labels)



In [111]:
resumen = cong.pivot_table(index='partido', columns='cat_ingreso', values='total_ingreso', aggfunc='mean')

In [112]:
resumen.columns = resumen.columns.astype(str) ### Tengo que convertir mis columnas, que eran "Categorical indexes" en strings (no siempre pasara esto, suele estar en string )
#resumen['total'] =resumen.sum(axis=1)

In [114]:
resumen = resumen.reset_index()

In [116]:
resumen_melted = resumen.melt(id_vars = 'partido', value_vars = ['cat_1', 'cat_2', 'cat_3', 'cat_4'])
resumen_melted

,partido,cat_ingreso,value
0,ACCION POPULAR,cat_1,3.369192e+03
1,ALIANZA PARA EL PROGRESO,cat_1,1.041231e+03
2,AVANZA PAIS - PARTIDO DE INTEGRACION SOCIAL,cat_1,9.412973e+02
3,DEMOCRACIA DIRECTA,cat_1,1.020975e+03
4,"EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD",cat_1,2.844792e+03
...,...,...,...
83,PODEMOS PERU,cat_4,6.274179e+05
84,RENACIMIENTO UNIDO NACIONAL,cat_4,8.229403e+05
85,RENOVACION POPULAR,cat_4,1.068811e+06
86,UNION POR EL PERU,cat_4,1.767411e+05
